In [1]:
from datetime import datetime
from typing import Tuple
from zipfile import ZipFile

from sklearn.utils.extmath import randomized_svd
import numpy as np
import pandas as pd
import plotly.express as px
from numpy.lib.recfunctions import merge_arrays
from sklearn.isotonic import IsotonicRegression

# Matched Pair Calibration on MovieLens

This notebook provides an example implementation of two different versions of Matched Pair Calibration (MPC):
- `compute_theory_mpc` - computes MPC according to Definition 3 in "Matched Pair Calibration for Ranking Fairness"
- `compute_practice_mpc` - computes MPC according to the additional practical concerns described in Section 4 of that paper.

To execute, download the [MovieLens 25M Dataset](https://grouplens.org/datasets/movielens/) and update the following configuration cell to indicate where it's located. The rest of the cells should succeed afterwards.

### Configuration

Modify the next cell for specifics of your run, notably the location of the zipped MovieLens 25M Dataset

In [2]:
# this should be a path to the movielens 25m zip file
MOVIELENS_LOCATION = "/tmp/ml-25m.zip"
TRAIN_TEST_SPLIT = 80  # 80% towards training
EMBEDDING_DIM = 64  # decomposes the training matrix as 64 dimensional representations
DEFAULT_CI = 0.95  # 95% confidence intervals
DEFAULT_MPC_PERC_THRESH = 0.01  # take the lowest 1% of possible pairs
DEFAULT_NUM_BOOTSTRAPS = 201  #  This should be greater than (1 - DEFAULT_CI) * 50

### Load Dataset

The next cells read the relevant datasets from MovieLens and create the train/test split for learning a ranking model and evaluating MPC on it.


In [4]:
with ZipFile(MOVIELENS_LOCATION) as zipf:
    with zipf.open("ml-25m/movies.csv") as moviesf:
        movies = pd.read_csv(moviesf)
    with zipf.open("ml-25m/ratings.csv") as ratingsf:
        ratings = pd.read_csv(ratingsf)

In [5]:
# filter out null timestamps and timestmaps before 2000 (since the dataset starts in 2006)
valid_dates = ratings.timestamp > datetime(2000, 1, 1).timestamp()
ratings = ratings[ratings.timestamp.notnull() & valid_dates]

In [6]:
time_split = np.percentile(ratings.timestamp, TRAIN_TEST_SPLIT)
training = ratings[ratings.timestamp < time_split]
testing = ratings[ratings.timestamp >= time_split]

### Training

These cells turn the training data into a matrix and computes a low-rank SVD decomposition storing the results in `movie_embeds` and `user_embeds` respectively.

In [7]:
# converts the training data to a matrix
# this can take a while as the matrix is very large
train_user_ids, user_inds = np.unique(training.userId, return_inverse=True)
train_movie_ids, movie_inds = np.unique(training.movieId, return_inverse=True)
training_matrix = np.full(
    (train_user_ids.size, train_movie_ids.size), training.rating.mean()
)
training_matrix[user_inds, movie_inds] = training.rating

In [8]:
# 64 dimensional embeddings, chosen arbitrarily
# since the matrix is large, this can take a while
# we use a seed for deterministic results
u, s, vt = randomized_svd(training_matrix, EMBEDDING_DIM, random_state=0)
user_embeds = u * np.sqrt(s)
movie_embeds = vt.T * np.sqrt(s)

### Evaluation Data

We collect the evaluation data into numpy arrays to make processing as fast as possible.

In [9]:
# join test data on movie genres
eval_data = testing.join(movies.set_index("movieId"), "movieId", how="inner")

In [10]:
# find examples in training data that have both user and movie embeddings, the rest will get filtered
user_inds = np.minimum(np.searchsorted(train_user_ids, eval_data.userId), train_user_ids.size - 1)
movie_inds = np.minimum(np.searchsorted(train_movie_ids, eval_data.movieId), train_movie_ids.size - 1)
mask = (train_user_ids[user_inds] == eval_data.userId) & (train_movie_ids[movie_inds] == eval_data.movieId)

In [11]:
# extract parallel arrays of all testing data
genre_set = {
    genre
    for genres in eval_data.genres[mask]
    for genre in genres.split("|")
    if genre != "(no genres listed)"
}
genres = np.array(sorted(genre_set), "U")  # all genres
batch_id = eval_data.userId.values[mask]  # "batch" ids, e.g. evaluation users
labels = eval_data.rating.values[mask]  # ratings
scores = np.sum(user_embeds[user_inds[mask]] * movie_embeds[movie_inds[mask]], 1)  # low rank predicted scores
groups = np.char.find(eval_data.genres.values[mask].astype("U")[:, None], genres).T >= 0  # boolean array of all genre memberships

## Calibration

Here we look at the post-hoc calibration on test data. Note, this is "cheating" in that we're calibrating on the test data itself, but this is also the best we could possibly do, e.g. represents optimal calibration, and the results show mpc errors still exist.

In [12]:
def calibrate(scores: np.ndarray, labels: np.ndarray) -> np.ndarray:
    """calibrates scores using isotonic regression"""
    model = IsotonicRegression()
    model.fit(scores, labels)
    return model.predict(scores)

In [13]:
def calibration(scores: np.ndarray, labs: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """get calibration estimate and 3x standard error for score buckets"""
    bins = np.concatenate([[-np.inf], np.linspace(0.75, 4.75, 9), [np.inf]])
    counts, _ = np.histogram(scores, bins)
    sums, _ = np.histogram(scores, bins, weights=labs)
    sumsq, _ = np.histogram(scores, bins, weights=labs**2)
    with np.errstate(all="ignore"):
        means = sums / counts
        var = sumsq / counts - means**2
        err = 3 * np.sqrt(var / counts)
    return means, err

In [62]:
# we can calibrate (note this is testing on training set, but serves our purpose fine)
frames = []
for genre, group in zip(genres, groups):
    gscores = scores[group]
    glabels = labels[group]
    means, err = calibration(gscores, glabels)
    frames.append(
        pd.DataFrame(
            {
                "score": np.linspace(0.5, 5, 10),
                "label": means,
                "label_err": err,
                "genre": [genre] * 10,
            }
        )
    )
px.line(
    pd.concat(frames, axis=0),
    x="score",
    y="label",
    error_y="label_err",
    color="genre",
    title="Ranking scores are roughly monotonic, but not calibrated",
    labels={"genre": "Genre", "label": "Average Rating", "score": "Ranking Score"},
    width=600,
    height=400,
)

In [64]:
# we can calibrate (note this is testing on training set, but serves our purpose fine)
frames = []
for genre, group in zip(genres, groups):
    gscores = scores[group]
    glabels = labels[group]
    calibrated = calibrate(gscores, glabels)
    means, err = calibration(calibrated, glabels)
    frames.append(
        pd.DataFrame(
            {
                "score": np.linspace(0.5, 5, 10),
                "label": means,
                "label_err": err,
                "genre": [genre] * 10,
            }
        )
    )
px.line(
    pd.concat(frames, axis=0),
    x="score",
    y="label",
    error_y="label_err",
    color="genre",
    title="Calibration works at calibrating scores",
    labels={"genre": "Genre", "label": "Average Rating", "score": "Calibrated Score"},
    width=600,
    height=400,
)

## Compute MPC

This goes through the training data and computes mpc errors for each group under various conditions: baseline mpc, boosted group, demoted group, and calibrated. For each condition, it also computes the NDCG. Note that for the baseline group, NDCG will be identical since it's the same ranking. The result is `mpc_data` that has all of the experimental results.

There a few helper functions to ease computation.

In [14]:
def boostrap_mean(
    values: np.ndarray, ci: float, num_boostraps: int = DEFAULT_NUM_BOOTSTRAPS
) -> Tuple[float, float]:
    """compute boostrap confidence intervals around the mean of an array

    This is similar to
    `scipy.stats.bootstrap(values, np.mean, confidence_level=ci, n_resamples=num_bootstraps, method='percentile')`
    but in general is a bit faster and works on the large arrays we use.

    Parameters
    ----------
    values : the values to bootstrap the mean of
    ci : the confidence interval in [0, 100]
    num_boostraps : the number of bootstraps to run for computing the ci
    """
    if values.size:
        bootstrap_counts = np.random.multinomial(
            values.size,
            np.full(values.size, 1 / values.size),
            num_boostraps,
        )
        bootstrap_weights = bootstrap_counts / values.size
        percs = [(1 - ci) / 2, (1 + ci) / 2]
        lower, upper = np.quantile(bootstrap_weights @ values, percs)
        return lower, upper
    else:
        return np.nan, np.nan

In [15]:
def batch_arange(lens: np.ndarray) -> np.ndarray:
    """compute the concatenation of several aranges
    
    This is identical to:
    ```
    np.concatenate([np.arange(i) for i in lens])
    ```
    but faster.
    """
    lens = lens[lens > 0]
    inds = lens.cumsum()
    diffs = np.ones(inds[-1], "i8")
    diffs[0] = 0
    diffs[inds[:-1]] -= lens[:-1]
    return diffs.cumsum()

In [16]:
def batch_pairs(inds: np.ndarray) -> np.ndarray:
    """compute all ways to choose pairs from the indices of a staggered batch array

    This is identical to
    ```
    np.concatenate([np.stack(np.triu_indices(i, 1)) + o for n, o in zip(np.diff(inds), inds[:-1])], 1)
    ```
    but faster.
    """
    lens = np.diff(inds)
    offsets = inds[:-1].repeat(lens * (lens - 1) // 2)
    n1 = lens - 1
    n1range = batch_arange(n1)
    minus = n1.repeat(n1) - n1range
    first = n1range.repeat(minus)
    second = batch_arange(minus) + first + 1
    first += offsets
    second += offsets
    return first, second

In [17]:
def compute_dcg(batch_id: np.ndarray, label: np.ndarray) -> np.ndarray:
    """compute the dcg of each batch given an array of batches and coresponding labels

    DCGs will be returned in sorted order by batch id
    """
    inds = np.flatnonzero(batch_id[:-1] != batch_id[1:])
    inds = np.insert(inds + 1, [0, inds.size], [0, batch_id.size])
    lens = np.diff(inds)

    discounts = np.log2(batch_arange(lens) + 2)
    return np.add.reduceat(label / discounts, inds[:-1])

In [18]:
def compute_ndcg(
    batch_id: np.ndarray,
    label: np.ndarray,
    score: np.ndarray,
    *,
    ci: float = DEFAULT_CI
) -> Tuple[float, float, float]:
    """compute average ndcg and confidence intervals

    NDCGs will be returned in sorted order by batch id
    """
    score_order = merge_arrays((batch_id, -score)).argsort(order=["f0", "f1"])
    dcg = compute_dcg(batch_id[score_order], label[score_order])
    label_order = merge_arrays((batch_id, -label)).argsort(order=["f0", "f1"])
    norm = compute_dcg(batch_id[label_order], label[label_order])

    ndcgs = dcg / norm
    avg = ndcgs.mean()
    lower, upper = boostrap_mean(ndcgs, ci)
    return lower, avg, upper

In [19]:
def compute_theory_mpc(
    batch_id: np.ndarray,
    labels: np.ndarray,
    scores: np.ndarray,
    group: np.ndarray,
    *,
    perc_thresh: float = DEFAULT_MPC_PERC_THRESH,
    ci: float = DEFAULT_CI,
) -> Tuple[float, float, float]:
    """theoretic mpc error for a set of data

    Computes forward MPC errors for pairs whose score difference is in the
    perc_thresh percentile, the first element doesn't belong to group, but
    the second element does

    Parameters
    ----------
    batch_id : array of batch identifiers, equal ids are in the same batch
    labels : array of true label values
    scores : array of predicted scores
    group : boolean array that's true for the subgroup to measure
    perc_thresh : the percentile threshold for similar scores
    ci : the width of the confidene threshold

    Returns
    -------
    lower : lower bound of confidence interval
    mpc : the expected mpc error, higher indicates more unfairness to the
      group in question, e.g. higher labels in expectations
    upper : upper bound of confidence interval
    """
    # order the batches so that items from the same batche are adjacent and
    # predictions are descending
    order = merge_arrays((batch_id, -scores)).argsort(order=["f0", "f1"])
    batch_id = batch_id[order]
    labels = labels[order]
    scores = scores[order]
    group = group[order]

    # compute all pairs pairs
    batches = batch_id[:-1] == batch_id[1:]
    inds = np.flatnonzero(~batches)
    finds, sinds = batch_pairs(np.insert(inds + 1, [0, inds.size], [0, batch_id.size]))

    # limit by group
    group_pairs = (~group[finds]) & (group[sinds])
    finds = finds[group_pairs]
    sinds = sinds[group_pairs]

    # find indices of perc_thresh lowest score differences
    score_diffs = scores[finds] - scores[sinds]
    part = int(score_diffs.size * perc_thresh)
    lowest = np.argpartition(score_diffs, part)[:part]

    # compute label differences for lowest score indices
    paired_label_diffs = labels[sinds[lowest]] - labels[finds[lowest]]

    # compute mpc
    mpc = paired_label_diffs.mean()
    lower, upper = boostrap_mean(paired_label_diffs, ci)
    return lower, mpc, upper

In [20]:
def compute_practice_mpc(
    batch_id: np.ndarray,
    labels: np.ndarray,
    scores: np.ndarray,
    group: np.ndarray,
    *,
    perc_thresh: float = DEFAULT_MPC_PERC_THRESH,
    ci: float = DEFAULT_CI,
) -> Tuple[float, float, float]:
    """practical mpc error for a set of data

    Computes MPC errors for pairs whos score difference is in the
    perc_thresh percentile.

    Parameters
    ----------
    batch_id : array of batch identifiers, equal ids are in the same batch
    labels : array of true label values
    scores : array of predicted scores
    group : boolean array that's true for the subgroup to measure
    perc_thresh : the percentile threshold for similar scores
    ci : the width of the confidene threshold

    Returns
    -------
    lower : lower bound of confidence interval
    mpc : the expected mpc error, higher indicates more unfairness to
      the group in question, e.g. higher labels in expectations
    upper : upper bound of confidence interval
    """
    # order the batches so that batches are adjacent and predictions are descending
    order = merge_arrays((batch_id, -scores)).argsort(order=["f0", "f1"])
    batch_id = batch_id[order]
    labels = labels[order]
    scores = scores[order]
    group = group[order]

    # compute batches
    pairs = (batch_id[:-1] == batch_id[1:]) & (group[:-1] != group[1:])

    # compute the percentile
    score_diffs = -np.diff(scores)
    thresh = np.quantile(score_diffs[pairs], perc_thresh)

    # compute matched pairs, and the corresponding label differences
    is_mp = pairs & (score_diffs <= thresh)
    signed_diffs = np.diff(labels) * np.where(group[1:], 1, -1)
    paired_label_diffs = signed_diffs[is_mp]

    # compute mpc
    mpc = paired_label_diffs.mean()
    lower, upper = boostrap_mean(paired_label_diffs, ci)
    return lower, mpc, upper

In [21]:
# compute mpc and ndcg for various treatment and genres
# this is computationally expensive with the bootstrapping, so it will take a while
frames = []
for genre, group in zip(genres, groups):
    # amount of boost
    boost = scores.std() / 3
    boosted = scores + boost * group
    demoted = scores - boost * group

    calibrated = np.empty_like(scores)
    calibrated[group] = calibrate(scores[group], labels[group])
    calibrated[~group] = calibrate(scores[~group], labels[~group])

    for name, treated in [
        ("Baseline", scores),
        ("Boosted", boosted),
        ("Demoted", demoted),
        ("Calibrated", calibrated),
    ]:
        tlower, tmpc, tupper = compute_theory_mpc(batch_id, labels, treated, group)
        plower, pmpc, pupper = compute_practice_mpc(batch_id, labels, treated, group)
        nlower, ndcg, nupper = compute_ndcg(batch_id, labels, treated)

        frames.append(
            pd.DataFrame(
                {
                    "genre": [genre],
                    "treatment": [name],
                    "theory_mpc": [tmpc],
                    "theory_lower": [tmpc - tlower],
                    "theory_upper": [tupper - tmpc],
                    "practice_mpc": [pmpc],
                    "practice_lower": [pmpc - plower],
                    "practice_upper": [pupper - pmpc],
                    "ndcg": [ndcg],
                    "ndcg_lower": [ndcg - nlower],
                    "ndcg_upper": [nupper - ndcg],
                }
            )
        )
    print("completed", genre)

mpc_data = pd.concat(frames, axis=0)

completed Action


completed Adventure


completed Animation


completed Children


completed Comedy


...

completed Romance


completed Sci-Fi


completed Thriller


completed War


completed Western


# Results

We now create plots and "tables" demonstrating all of the results.

This creates the LaTeX for the results table.

In [22]:
table = (
    mpc_data[mpc_data.genre == "Documentary"]
    .set_index("treatment")
    .reindex(["Boosted", "Baseline", "Demoted", "Calibrated"])
)
print(
    pd.concat(
        [
            table.theory_mpc.combine(
                np.maximum(table.theory_lower, table.theory_upper),
                lambda a, b: f"${a:.3f} \pm {b:.3f}$",
            ).rename("MPC"),
            table.ndcg.combine(
                np.maximum(table.ndcg_lower, table.ndcg_upper),
                lambda a, b: f"${a:.4f} \pm {b:.4f}$",
            ).rename("NDCG"),
        ],
        axis=1,
    )
    .reset_index()
    .rename(columns={"treatment": "Scoring Rule"})
    .to_latex(escape=False, index=False)
)

\begin{tabular}{lll}
\toprule
Scoring Rule &                MPC &                 NDCG \\
\midrule
     Boosted &  $0.006 \pm 0.020$ &  $0.9592 \pm 0.0008$ \\
    Baseline &  $0.278 \pm 0.011$ &  $0.9590 \pm 0.0008$ \\
     Demoted &  $0.507 \pm 0.007$ &  $0.9587 \pm 0.0009$ \\
  Calibrated &  $0.066 \pm 0.017$ &  $0.9595 \pm 0.0008$ \\
\bottomrule
\end{tabular}



### Theory Version

First we plot results for the theoretic implementation of MPC that are implemented in the paper.

In [23]:
theory_ordered = pd.concat(
    sorted(
        (f for _, f in mpc_data.groupby("genre")),
        key=lambda x: x[x.treatment == "Baseline"].theory_mpc.item(),
    )
)

In [59]:
treatments = ["Boosted", "Baseline", "Demoted"]
px.bar(
    theory_ordered[theory_ordered.treatment.isin(treatments)],
    y="genre",
    x="theory_mpc",
    error_x="theory_upper",
    error_x_minus="theory_lower",
    color="treatment",
    barmode="group",
    category_orders={"treatment": treatments},
    title="Additive boosts have expected directional effects on Matched Pair Calibration",
    labels={
        "genre": "Genre",
        "theory_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=700,
)

In [66]:
treatments = ["Calibrated", "Baseline"]
px.bar(
    theory_ordered[theory_ordered.treatment.isin(treatments)],
    y="genre",
    x="theory_mpc",
    error_x="theory_upper",
    error_x_minus="theory_lower",
    color="treatment",
    barmode="group",
    category_orders={"treatment": treatments},
    title="Group Calibration does not remove Matched Pair Calibration errors",
    labels={
        "genre": "Genre",
        "theory_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=600,
)

### Practice Version

Now we plots the same charts for the version of MPC that often makes more sense in practice. The qualitative results are identical.

In [28]:
practice_ordered = pd.concat(
    sorted(
        (f for _, f in mpc_data.groupby("genre")),
        key=lambda x: x[x.treatment == "Baseline"].practice_mpc.item(),
    )
)

In [56]:
treatments = ["Boosted", "Baseline", "Demoted"]
px.bar(
    practice_ordered[practice_ordered.treatment.isin(treatments)],
    y="genre",
    x="practice_mpc",
    error_x="practice_upper",
    error_x_minus="practice_lower",
    color="treatment",
    barmode="group",
    category_orders={"treatment": treatments},
    title="Additive boosts have expected directional effects on practical Matched Pair Calibration",
    labels={"genre": "Genre", "practice_mpc": "Matched Pair Calibration Error", "treatment": "Scoring Rule"},
    width=600,
    height=700,
)

In [51]:
treatments = ["Baseline", "Calibrated"]
px.bar(
    practice_ordered[practice_ordered.treatment.isin(treatments)],
    y="genre",
    x="practice_mpc",
    error_x="practice_upper",
    error_x_minus="practice_lower",
    color="treatment",
    barmode="group",
    category_orders={"treatment": treatments},
    title="Group Calibration does not remove practical Matched Pair Calibration errors",
    labels={
        "genre": "Genre",
        "practice_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=600,
)

In [48]:
px.bar(
    mpc_data,
    y="genre",
    x="ndcg",
    error_x="ndcg_upper",
    error_x_minus="ndcg_lower",
    range_x=(0.955, 0.961),
    color="treatment",
    barmode="group",
    category_orders={"treatment": ["Calibrated", "Demoted", "Baseline", "Boosted"]},
    title="NDCG is high and nearly identical for most scoring rules, but a few lower it significantly",
    labels={
        "genre": "Genre",
        "theory_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=900,
)

In [45]:
px.scatter(
    mpc_data,
    x="theory_mpc",
    y="ndcg",
    color="treatment",
    error_x="theory_upper",
    error_x_minus="theory_lower",
    error_y="ndcg_upper",
    error_y_minus="ndcg_lower",
    title="High magnitude MPC Errors correlate with lower NDCG indicating misranking",
    labels={
        "ndcg": "NDCG",
        "theory_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=400,
)

In [44]:
px.scatter(
    mpc_data,
    x="practice_mpc",
    y="ndcg",
    color="treatment",
    error_x="practice_upper",
    error_x_minus="practice_lower",
    error_y="ndcg_upper",
    error_y_minus="ndcg_lower",
    title="High magnitude practice MPC Errors correlate with lower NDCG indicating misranking",
    labels={
        "ndcg": "NDCG",
        "practice_mpc": "Matched Pair Calibration Error",
        "treatment": "Scoring Rule",
    },
    width=600,
    height=400,
)